# 6.14.4 Проект - решение


In [ ]:
import asyncio
import ast
import re
import urllib.request
from typing import Any, Dict, List

RAW_URL = "https://raw.githubusercontent.com/nefelsay/asyncio/main/bus_passengers.py"

def load_passengers() -> List[Dict[str, Any]]:
    with urllib.request.urlopen(RAW_URL, timeout=30) as resp:
        text = resp.read().decode("utf-8")

    m = re.search(r"passengers\s*=\s*(\[[\s\S]*\])", text)
    if not m:
        raise ValueError("Не удалось найти список passengers в bus_passengers.py")
    return ast.literal_eval(m.group(1))

async def board_passenger(p: Dict[str, Any]) -> str:
    name = p["Name"]
    speed = int(p["Speed"])
    await asyncio.sleep(speed)
    return name

async def main():
    passengers = load_passengers()
    tasks = [asyncio.create_task(board_passenger(p)) for p in passengers]

    try:
        await asyncio.wait_for(asyncio.gather(*tasks), timeout=5)
    except asyncio.TimeoutError:
        for t in tasks:
            if not t.done():
                t.cancel()
        await asyncio.gather(*tasks, return_exceptions=True)

    for p, t in zip(passengers, tasks):
        name = p["Name"]
        job = p["Job"]
        if not t.cancelled():
            print(f"{name} сел в автобус.")
        else:
            print(f"{name} {job} не успел/а вовремя сесть в автобус.")

asyncio.run(main())
